In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    ManagedOnlineEndpoint,
    Model,
    CodeConfiguration,
    Environment,
    ManagedIdentityConfiguration,
    IdentityConfiguration
)

import random

In [2]:
# Create handle to ml workspace
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

Found the config file in: /config.json
Class DeploymentTemplateOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [3]:
rand = random.randint(0, 10000)
endpoint_name = f"endpt-moe-{rand}"
print(endpoint_name)

endpt-moe-7670


In [4]:
## UAI resource details - ARM id
uai_resource_id = "/subscriptions/{subid}/resourceGroups/{resourcegroup-name}/providers/Microsoft.ManagedIdentity/userAssignedIdentities/{uai-resource-name}"

In [5]:
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name, 
    auth_mode="key",
    identity=IdentityConfiguration(
        type="UserAssigned",
        user_assigned_identities=[
            ManagedIdentityConfiguration(
                resource_id=uai_resource_id
            )
        ]
    )
)

In [ ]:
# create endpoint
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [ ]:
# print endpoint identity details
## endpoint's identity is already given blob reader permission on the default storage account
endpoint = ml_client.online_endpoints.get(endpoint_name)
print(endpoint.identity.type)
print(endpoint.identity.user_assigned_identities[0].client_id)

## Remember to assign for this ID (manual step) - as its user assigned identity based managed endpoint
#   - blob contributor role on storage account
#   - acrpull role on the acr

user_assigned
ec8a73d0-7af8-41e9-bd68-85671d226c8c


In [10]:
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=Model(path="./model/"),
    code_configuration=CodeConfiguration(
        code="./onlinescoring/", scoring_script="score_uai.py"
    ),
    environment=Environment(
        conda_file="./environment/conda-managedidentity.yml",
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
    environment_variables={
        "UAI_CLIENT_ID": endpoint.identity.user_assigned_identities[0].client_id,
    },
)

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

In [ ]:
# set traffic to 100% for deployment
endpoint.traffic = {str(deployment.name): 100}
ml_client.begin_create_or_update(endpoint).result()

In [13]:
# test endpoint
sample_data = "./invoke/sample-request.json"
ml_client.online_endpoints.invoke(endpoint_name=endpoint_name, request_file=sample_data)

'[11055.977245525679, 4503.079536107787]'